In [1]:
from gensim.models import FastText as ft
from __future__ import print_function

In [2]:
from gensim.models import FastText as ft
from __future__ import print_function
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
data = pd.read_pickle('seoul_oneroom_for_ml.pkl')

In [54]:
with open('seoul_oneroom.pickle', 'wb') as fw:
    pickle.dump(data,fw)

In [44]:
data

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,token,text_noun
0,34134003,38.077637,128.618335,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,강원도 양양군 양양읍,신축첫입주 전세 월세 신축첫입주입니다\r\n월세는 2층\r\n전세는 3층부터입니다\...,https://ic.zigbang.com/ic/items/34134003/1.jpg,전세,8000,NaN,7.0,강원도 양양군 양양읍 신축첫입주 전세 월세 신축첫입주입니다 월세는 층 전세는 층부터...,"[강원도, 양양군, 양양읍, 신축, 입주, 전세, 월세, 신축, 입, 주입, 다, ...","[강원도, 양양군, 양양읍, 신축, 입주, 전세, 월세, 신축, 입, 주입, 월세,..."
1,34153813,38.077641,128.618986,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,강원도 양양군 양양읍,1.5룸 큰원룸 풀옵션 신축첫입주 신축첫입주입니다\r\n\r\n전세는 3층 부터입니...,https://ic.zigbang.com/ic/items/34153813/1.jpg,전세,10000,NaN,10.0,강원도 양양군 양양읍 룸 큰원룸 풀옵션 신축첫입주 신축첫입주입니다 전세는 층 부터입...,"[강원도, 양양군, 양양읍, 룸, 크다, 원룸, 풀, 옵션, 신축, 입주, 신축, ...","[강원도, 양양군, 양양읍, 룸, 원룸, 풀, 옵션, 신축, 입주, 신축, 입, 주..."
2,34273845,38.184565,128.601096,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,강원도 속초시 조양동,동해바다가 잘보이는 방1 거실1 구조 전입신고 x\r\n동해바다 속초해수욕장 대포항...,https://ic.zigbang.com/ic/items/34273845/1.jpg,월세,500,70.0,15.0,강원도 속초시 조양동 동해바다가 잘보이는 방 거실 구조 전입신고 동해바다 속초해수욕...,"[강원도, 속초시, 조양동, 동해, 바다, 보이다, 방, 거실, 구조, 전입, 신고...","[강원도, 속초시, 조양동, 동해, 바다, 방, 거실, 구조, 전입, 신고, 동해,..."
3,34252466,38.186523,128.598820,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,강원도 속초시 조양동,조양동 분리형원룸 조양동 분리형원룸\r\n교통과 인프라가 잘갖춰져있으며\r\n주택단...,https://ic.zigbang.com/ic/items/34252466/1.jpg,월세,500,40.0,7.0,강원도 속초시 조양동 조양동 분리형원룸 조양동 분리형원룸 교통과 인프라가 잘갖춰져있...,"[강원도, 속초시, 조양동, 조양동, 분리, 원룸, 조양동, 분리, 원룸, 교통, ...","[강원도, 속초시, 조양동, 조양동, 분리, 원룸, 조양동, 분리, 원룸, 교통, ..."
4,34302547,38.187599,128.594645,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,강원도 속초시 조양동,조양동 신축 풀옵션 분리형 원룸 신축건물입니다\r\n반려동물 사육불가능 합니다\r\...,https://ic.zigbang.com/ic/items/34302547/1.jpg,월세,1000,55.0,5.0,강원도 속초시 조양동 조양동 신축 풀옵션 분리형 원룸 신축건물입니다 반려동물 사육불...,"[강원도, 속초시, 조양동, 조양동, 신축, 풀, 옵션, 분리, 원룸, 신축, 건물...","[강원도, 속초시, 조양동, 조양동, 신축, 풀, 옵션, 분리, 원룸, 신축, 건물..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102281,34203993,33.538358,126.664236,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,제주도 제주시 조천읍,⭐조천읍 함덕리 반려묘 가능한 분리형 원룸 전세⭐ ❤제주 핫한 BEST방❤\r\n\...,https://ic.zigbang.com/ic/items/34203993/1.jpg,전세,10000,NaN,5.0,제주도 제주시 조천읍 조천읍 함덕리 반려묘 가능한 분리형 원룸 전세 제주 핫한 방 ...,"[제주도, 제주시, 조천읍, 조천읍, 함덕, 리, 반려, 묘, 분리, 원룸, 전세,...","[제주도, 제주시, 조천읍, 조천읍, 함덕, 리, 반려, 묘, 분리, 원룸, 전세,..."
102282,34309621,33.537858,126.663626,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,제주도 제주시 조천읍,🍒🍒테라스있는오픈형원룸&풀옵션 🍒🍒테라스있는오픈형원룸&풀옵션\r\n\r\n전세대출가...,https://ic.zigbang.com/ic/items/34309621/1.jpg,전세,9000,NaN,3.0,제주도 제주시 조천읍 테라스있는오픈형원룸 풀옵션 테라스있는오픈형원룸 풀옵션 전세대출...,"[제주도, 제주시, 조천읍, 테라스, 오픈, 원룸, 풀, 옵션, 테라스, 오픈, 원...","[제주도, 제주시, 조천읍, 테라스, 오픈, 원룸, 풀, 옵션, 테라스, 오픈, 원..."
102283,34269795,33.540079,126.676017,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,제주도 제주시 조천읍,🍒🍒함덕해수욕장인근&풀옵션원룸&반려동물X 🍒🍒함덕해수욕장인근&풀옵션원룸&반려동물X\...,https://ic.zigbang.com/ic/items/34269795/1.jpg,월세,100,54.0,1.0,제주도 제주시 조천읍 함덕해수욕장인근 풀옵션원룸 반려동물 함덕해수욕장인근 풀옵션원룸...,"[제주도, 제주시, 조천읍, 함덕, 해수욕장, 인근, 풀, 옵션, 원룸, 반려동물,...","[제주도, 제주시, 조천읍, 함덕, 해수욕장, 인근, 풀, 옵션, 원룸, 반려동물,..."
102284,34296056,33.541841,126.662953,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,제주도 제주시 조천읍,++제주시 함덕 해수욕장 부근 깔끔한 원룸 매물++ ✔상세설명✔\r\n\r\n \...,https://ic.zigbang.com/ic/items/34296056/1.jpg,월세,100,60.0,5.0,제주도 제주시 조천읍 제주시 함덕 해수욕장 부근 깔끔한 원룸 매물 상세설명 넓고 아...,"[제주도, 제주시, 조천읍, 제주시, 함덕, 해수욕장, 부근, 원룸, 매물, 상세,...","[제주도, 제주시, 조천읍, 제주시, 함덕, 해수욕장, 부근, 원룸, 매물, 상세,..."


In [5]:
result = data.token

In [6]:
model = ft(result, vector_size=300, window=5, min_count=5, workers=4, sg=1)

In [7]:
model.wv['서울']

array([-0.29337972,  0.14440598,  0.10736462, -0.07455414, -0.06940249,
       -0.1658309 , -0.16887757,  0.08936661, -0.263283  ,  0.24901536,
        0.10807529,  0.03078796,  0.22780228,  0.43537953,  0.28253934,
        0.02482133, -0.0915461 , -0.15131097,  0.0625412 , -0.18511832,
       -0.19026709, -0.17938882,  0.01992948,  0.15816773, -0.15290405,
       -0.21219385,  0.20539953,  0.01616549,  0.38792807, -0.20945664,
        0.53165245,  0.42726094,  0.03063033, -0.10712961,  0.47776553,
       -0.15523943, -0.01445072, -0.22303054,  0.6286668 ,  0.09228489,
        0.39069685, -0.4256132 , -0.22565758, -0.16445419,  0.08601354,
       -0.30430788, -0.4005518 , -0.25726554,  0.05569322,  0.3574834 ,
       -0.052287  , -0.30352676, -0.00893816,  0.14031827, -0.3883325 ,
       -0.02901534,  0.05191518,  0.12530755, -0.3082931 ,  0.32479775,
       -0.01544101, -0.6028981 ,  0.2028205 ,  0.26515228,  0.06791268,
        0.2647398 ,  0.20313835,  0.07030215, -0.24907403, -0.03

In [45]:
import pickle

In [15]:
with open('rec_model.pickle', 'wb') as fw:
    pickle.dump(model,fw)

In [12]:
model.wv.most_similar('화곡역 근처 엘리베이터 있고 반려동물 가능한 집')

[('엘리베이터', 0.7637404799461365),
 ('화곡역', 0.5632096529006958),
 ('반려동물', 0.5401337146759033),
 ('사초', 0.500750720500946),
 ('엘유', 0.48255160450935364),
 ('지정석', 0.48031970858573914),
 ('엘베', 0.47603535652160645),
 ('년산', 0.4747368395328522),
 ('까치산역', 0.4743444621562958),
 ('신목동역', 0.46470752358436584)]

In [49]:
def sentence_vectorizer(input_sentence):
    imsi = None
    for i in input_sentence:
        if imsi is None:
            imsi = i
        else:
            imsi += i
    

    return imsi / len(input_sentence)

In [50]:
sentence_vectors = []

for i in data['token']:
    sentence_vectors.append(sentence_vectorizer(model.wv[i]))

In [10]:
len(sentence_vectors)

102286

In [51]:
data['vector'] = sentence_vectors

In [52]:
data.reset_index(inplace=True)

In [53]:
data.drop(['index'], axis=1, inplace = True)

In [14]:
print('문서 벡터의 수 :',len(sentence_vectors))

문서 벡터의 수 : 102286


In [15]:
data

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,token,text_noun,vector
0,34134003,38.077637,128.618335,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,강원도 양양군 양양읍,신축첫입주 전세 월세 신축첫입주입니다\r\n월세는 2층\r\n전세는 3층부터입니다\...,https://ic.zigbang.com/ic/items/34134003/1.jpg,전세,8000,NaN,7.0,강원도 양양군 양양읍 신축첫입주 전세 월세 신축첫입주입니다 월세는 층 전세는 층부터...,"[강원도, 양양군, 양양읍, 신축, 입주, 전세, 월세, 신축, 입, 주입, 다, ...","[강원도, 양양군, 양양읍, 신축, 입주, 전세, 월세, 신축, 입, 주입, 월세,...","[-0.11124457, -0.03362075, -0.1093692, 0.06558..."
1,34153813,38.077641,128.618986,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,강원도 양양군 양양읍,1.5룸 큰원룸 풀옵션 신축첫입주 신축첫입주입니다\r\n\r\n전세는 3층 부터입니...,https://ic.zigbang.com/ic/items/34153813/1.jpg,전세,10000,NaN,10.0,강원도 양양군 양양읍 룸 큰원룸 풀옵션 신축첫입주 신축첫입주입니다 전세는 층 부터입...,"[강원도, 양양군, 양양읍, 룸, 크다, 원룸, 풀, 옵션, 신축, 입주, 신축, ...","[강원도, 양양군, 양양읍, 룸, 원룸, 풀, 옵션, 신축, 입주, 신축, 입, 주...","[-0.06148533, -0.045128807, -0.10411783, 0.093..."
2,34273845,38.184565,128.601096,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,강원도 속초시 조양동,동해바다가 잘보이는 방1 거실1 구조 전입신고 x\r\n동해바다 속초해수욕장 대포항...,https://ic.zigbang.com/ic/items/34273845/1.jpg,월세,500,70.0,15.0,강원도 속초시 조양동 동해바다가 잘보이는 방 거실 구조 전입신고 동해바다 속초해수욕...,"[강원도, 속초시, 조양동, 동해, 바다, 보이다, 방, 거실, 구조, 전입, 신고...","[강원도, 속초시, 조양동, 동해, 바다, 방, 거실, 구조, 전입, 신고, 동해,...","[-0.1194009, -0.01392976, -0.042718634, 0.0447..."
3,34252466,38.186523,128.598820,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,강원도 속초시 조양동,조양동 분리형원룸 조양동 분리형원룸\r\n교통과 인프라가 잘갖춰져있으며\r\n주택단...,https://ic.zigbang.com/ic/items/34252466/1.jpg,월세,500,40.0,7.0,강원도 속초시 조양동 조양동 분리형원룸 조양동 분리형원룸 교통과 인프라가 잘갖춰져있...,"[강원도, 속초시, 조양동, 조양동, 분리, 원룸, 조양동, 분리, 원룸, 교통, ...","[강원도, 속초시, 조양동, 조양동, 분리, 원룸, 조양동, 분리, 원룸, 교통, ...","[-0.10207352, -0.010908589, -0.055541266, 0.08..."
4,34302547,38.187599,128.594645,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,강원도 속초시 조양동,조양동 신축 풀옵션 분리형 원룸 신축건물입니다\r\n반려동물 사육불가능 합니다\r\...,https://ic.zigbang.com/ic/items/34302547/1.jpg,월세,1000,55.0,5.0,강원도 속초시 조양동 조양동 신축 풀옵션 분리형 원룸 신축건물입니다 반려동물 사육불...,"[강원도, 속초시, 조양동, 조양동, 신축, 풀, 옵션, 분리, 원룸, 신축, 건물...","[강원도, 속초시, 조양동, 조양동, 신축, 풀, 옵션, 분리, 원룸, 신축, 건물...","[-0.054081187, -0.04620606, -0.05409909, 0.123..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102281,34203993,33.538358,126.664236,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,제주도 제주시 조천읍,⭐조천읍 함덕리 반려묘 가능한 분리형 원룸 전세⭐ ❤제주 핫한 BEST방❤\r\n\...,https://ic.zigbang.com/ic/items/34203993/1.jpg,전세,10000,NaN,5.0,제주도 제주시 조천읍 조천읍 함덕리 반려묘 가능한 분리형 원룸 전세 제주 핫한 방 ...,"[제주도, 제주시, 조천읍, 조천읍, 함덕, 리, 반려, 묘, 분리, 원룸, 전세,...","[제주도, 제주시, 조천읍, 조천읍, 함덕, 리, 반려, 묘, 분리, 원룸, 전세,...","[-0.10914681, -0.084192894, -0.07113353, 0.080..."
102282,34309621,33.537858,126.663626,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,제주도 제주시 조천읍,🍒🍒테라스있는오픈형원룸&풀옵션 🍒🍒테라스있는오픈형원룸&풀옵션\r\n\r\n전세대출가...,https://ic.zigbang.com/ic/items/34309621/1.jpg,전세,9000,NaN,3.0,제주도 제주시 조천읍 테라스있는오픈형원룸 풀옵션 테라스있는오픈형원룸 풀옵션 전세대출...,"[제주도, 제주시, 조천읍, 테라스, 오픈, 원룸, 풀, 옵션, 테라스, 오픈, 원...","[제주도, 제주시, 조천읍, 테라스, 오픈, 원룸, 풀, 옵션, 테라스, 오픈, 원...","[-0.07035034, -0.06311317, -0.07293233, 0.0281..."
102283,34269795,33.540079,126.676017,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,제주도 제주시 조천읍,🍒🍒함덕해수욕장인근&풀옵션원룸&반려동물X 🍒🍒함덕해수욕장인근&풀옵션원룸&반려동물X\...,https://ic.zigbang.com/ic/items/34269795/1.jpg,월세,100,54.0,1.0,제주도 제주시 조천읍 함덕해수욕장인근 풀옵션원룸 반려동물 함덕해수욕장인근 풀옵션원룸...,"[제주도, 제주시, 조천읍, 함덕, 해수욕장, 인근, 풀, 옵션, 원룸, 반려동물,...","[제주도, 제주시, 조천읍, 함덕, 해수욕장, 인근, 풀, 옵션, 원룸, 반려동물,...","[-0.03120451, -0.022195337, -0.04322323, 0.037..."
102284,34296056,33.541841,126.662953,{'url': 'https://www.zigbang.com/home/oneroom/...,https://ww

In [16]:
data.drop(['token','text_noun'], axis=1, inplace=True)

In [18]:
data.drop(['doc'], axis=1, inplace=True)

In [28]:
data.to_csv('seoul_room_for_db.csv', encoding='utf-8-sig')

In [25]:
data.detail = data.detail.str.replace("\n", "\\\n")

In [26]:
data.detail

0         신축첫입주 전세 월세 신축첫입주입니다\r\\n월세는 2층\r\\n전세는 3층부터입니...
1         1.5룸 큰원룸 풀옵션 신축첫입주 신축첫입주입니다\r\\n\r\\n전세는 3층 부터...
2         동해바다가 잘보이는 방1 거실1 구조 전입신고 x\r\\n동해바다 속초해수욕장 대포...
3         조양동 분리형원룸 조양동 분리형원룸\r\\n교통과 인프라가 잘갖춰져있으며\r\\n주...
4         조양동 신축 풀옵션 분리형 원룸 신축건물입니다\r\\n반려동물 사육불가능 합니다\r...
                                ...                        
102281    ⭐조천읍 함덕리 반려묘 가능한 분리형 원룸 전세⭐ ❤제주 핫한 BEST방❤\r\\n...
102282    🍒🍒테라스있는오픈형원룸&풀옵션 🍒🍒테라스있는오픈형원룸&풀옵션\r\\n\r\\n전세대...
102283    🍒🍒함덕해수욕장인근&풀옵션원룸&반려동물X 🍒🍒함덕해수욕장인근&풀옵션원룸&반려동물X\...
102284    ++제주시 함덕 해수욕장 부근 깔끔한 원룸  매물++ ✔상세설명✔\r\\n\r\\n...
102285    💙💚💛❤함덕바다 뷰 깔끔한 원룸❤💜🖤💖 ❤제주 핫한 BEST방❤\r\\n\r\\n ...
Name: detail, Length: 102286, dtype: object

In [18]:
cosine_similarities = cosine_similarity(sentence_vectors, sentence_vectors)

In [19]:
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (102286, 102286)


In [15]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

In [16]:
from konlpy.tag import Okt
from tqdm import tqdm

# 입력한 조건과 가장 비슷한 매물 추천

In [22]:
# 비슷한 매물 상위 20개 추천 cosine similarity
input_data = input('찾고자 하는 매물의 정보를 입력해주세요.:')

input_vector = model.wv[input_data]
cosine_similarities = cosine_similarity(sentence_vectors, [input_vector])
sim_scores = list(enumerate(cosine_similarities))
sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)

rec_index = [i[0] for i in sim_scores]
rec_index = rec_index[:20]

recommend = data.iloc[rec_index]
recommend

찾고자 하는 매물의 정보를 입력해주세요.:ㅎㅎ


,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,token,text_noun,vector
59011,33912256,37.560637,126.827830,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/339...,서울시 강서구 마곡동,"치명적인 매력을 갖고있는 현대힐스테이트에코마곡역! 전용면적 , 공급면적 : 전용면...",https://ic.zigbang.com/ic/items/33912256/1.jpg,전세,17000,NaN,7.0,서울시 강서구 마곡동 치명적인 매력을 갖고있는 현대힐스테이트에코마곡역 전용면적 공급...,"[서울시, 강서구, 마, 곡동, 치명, 매력, 갖다, 현대, 힐스테이트, 에코, 마...","[서울시, 강서구, 마, 곡동, 치명, 매력, 현대, 힐스테이트, 에코, 마곡역, ...","[-0.024995266, 0.005671774, -0.18028872, 0.179..."
22175,34244625,37.492423,126.992441,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,서울시 서초구 방배동,🟦🟧🟥🟩신축 고급빌라 고품격자재 주거환경 신축 방배동 고급빌라 \r\n\r\n고품격...,https://ic.zigbang.com/ic/items/34244625/1.jpg,전세,200000,NaN,25.0,서울시 서초구 방배동 신축 고급빌라 고품격자재 주거환경 신축 방배동 고급빌라 고품격...,"[서울시, 서초구, 방배동, 신축, 고급, 빌라, 고품격, 자재, 주거, 환경, 신...","[서울시, 서초구, 방배동, 신축, 고급, 빌라, 고품격, 자재, 주거, 환경, 신...","[-0.07647773, -0.009555526, -0.16504514, 0.105..."
52711,34218460,37.496226,126.841281,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,서울시 구로구 오류동,신축빌라 처음 입주 하십니다. 좋아요 신축빌라 처음 입주 하십니다. 좋아요\r\...,https://ic.zigbang.com/ic/items/34218460/1.jpg,월세,5000,100.0,6.0,서울시 구로구 오류동 신축빌라 처음 입주 하십니다 좋아요 신축빌라 처음 입주 하십니...,"[서울시, 구로구, 오류동, 신축, 빌라, 처음, 신축, 빌라, 처음, 주차, 신축...","[서울시, 구로구, 오류동, 신축, 빌라, 처음, 신축, 빌라, 처음, 주차, 신축...","[0.09212152, -0.046329718, -0.24119923, 0.0880..."
61727,34205188,37.640373,126.620908,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,경기도 김포시 구래동,"🍎신축(●'◡'●)분리형 1.5룸,즉시입주🍎블라인드지원 \r\n안녕하세요^^\r\n...",https://ic.zigbang.com/ic/items/34205188/1.jpg,월세,500,45.0,8.0,경기도 김포시 구래동 신축 분리형 룸 즉시입주 블라인드지원 안녕하세요 가장 힘들다는...,"[경기도, 김포시, 구래동, 신축, 분리, 룸, 블라인드, 지원, 가장, 보금자리,...","[경기도, 김포시, 구래동, 신축, 분리, 룸, 블라인드, 지원, 가장, 보금자리,...","[-0.05915387, -0.07060484, -0.13223438, 0.0887..."
59014,34190055,37.560648,126.827846,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,서울시 강서구 마곡동,"마곡에서 살기좋은 오피스텔 Top3안에드는 에코마곡역~ 전용면적 , 공급면적 : ...",https://ic.zigbang.com/ic/items/34190055/1.jpg,전세,17500,NaN,7.0,서울시 강서구 마곡동 마곡에서 살기좋은 오피스텔 안에드는 에코마곡역 전용면적 공급면...,"[서울시, 강서구, 마, 곡동, 마곡, 살기, 오피스텔, 에드, 에코, 마곡역, 전...","[서울시, 강서구, 마, 곡동, 마곡, 살기, 오피스텔, 에드, 에코, 마곡역, 전...","[-0.02405414, 0.037051123, -0.17905268, 0.1513..."
58968,34025236,37.559707,126.828867,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/340...,서울시 강서구 마곡동,"❤️❤️분위기에취한다~ ❤️에코동익❤️ 전용면적 , 공급면적 : 전용면적23.4...",https://ic.zigbang.com/ic/items/34025236/1.jpg,전세,20000,NaN,7.0,서울시 강서구 마곡동 분위기에취한다 에코동익 전용면적 공급면적 전용면적 공급면적 중...,"[서울시, 강서구, 마, 곡동, 분위기, 취한, 다, 에코, 익, 전용, 면적, 공...","[서울시, 강서구, 마, 곡동, 분위기, 취한, 에코, 익, 전용, 면적, 공급, ...","[-0.016967906, 0.009629265, -0.1963747, 0.1606..."
57821,34305739,37.547272,126.877198,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,서울시 강서구 염창동,급행염창역 도보 3분 층고 높은 복층형오피스텔 ** 염창역 도보 3분 층고 높은 남...,https://ic.zigbang.com/ic/items/34305739/1.jpg,월세,1000,63.0,8.0,서울시 강서구 염창동 급행염창역 도보 분 층고 높은 복층형오피스텔 염창역 도보 분 ...,"[서울시, 강서구, 염창동, 급행, 염창역, 도보, 층, 복층, 오피스텔, 염창역,...","[서울시, 강서구, 염창동, 급행, 염창역, 도보, 층, 복층, 오피스텔, 염창역,...","[-0.15771823, 0.058077324, -0.06312988, -0.003..."
61758,34309305,37.640373,126.620862,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,경기도 김포시 구래동,💘💘💘신축 즉시입주가💕입주청소 및 블라인드 지원 안녕하세요^^\r\n가장 힘들다는 ...,https://ic.zigbang.com/ic/items/34309305/1.jpg,월세,400,45.0,8.0,경기도 김포시 구래동 신축 즉시입주가 입주청소 및 블라인드 지원 안녕하세요 가장 힘...,"[경기도, 김포시, 구래동, 신축, 입, 주가, 청소, 블라인드, 지원, 가장, 보...","[경기도, 김포시, 구래동, 신축, 입, 주가, 청소, 블라인드, 지원, 가장, 보...","[-0.049994458, -0.075651065, -0.12959853, 0.08..."
59016,34300092,37.560657,126.827190,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,서울시 강서

In [37]:
# Euclidean distance
input_data = input('찾고자 하는 매물의 정보를 입력해주세요.:')

input_vector = model.wv[input_data]
ud = euclidean_distances(sentence_vectors, [input_vector])
sim_scores = list(enumerate(ud))
sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = False)

rec_index = [i[0] for i in sim_scores]
rec_index = rec_index[:20]

recommend = data.iloc[rec_index]
recommend

찾고자 하는 매물의 정보를 입력해주세요.:협재해수욕장


,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,token,text_noun,vector
77941,34117502,35.164747,129.161365,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,부산광역시 해운대구 우동,"더블역세권,분리형 방1,거1, 바닷가 도보권, 최상의입지. 해운대역 중동역 더블역세...",https://ic.zigbang.com/ic/items/34117502/1.jpg,월세,1000,60.0,10.0,부산광역시 해운대구 우동 더블역세권 분리형 방 거 바닷가 도보권 최상의입지 해운대역...,"[부산광역시, 해운대구, 우동, 더블, 권, 분리, 방, 바닷가, 도보, 최상, 입...","[부산광역시, 해운대구, 우동, 더블, 권, 분리, 방, 바닷가, 도보, 최상, 입...","[-0.0222658, -0.054434765, 0.06747609, 0.03954..."
78153,34313178,35.168066,129.178447,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,부산광역시 해운대구 좌동,"해운대애드웰오피스텔입니다.전입불가 ♥실사진, 장산역 분♥\r\n\r\n오피스텔 66...",https://ic.zigbang.com/ic/items/34313178/1.jpg,월세,1000,99.0,13.0,부산광역시 해운대구 좌동 해운대애드웰오피스텔입니다 전입불가 실사진 장산역 분 오피스...,"[부산광역시, 해운대구, 좌동, 해운대, 애, 드웰, 오피스텔, 전입, 불가, 실,...","[부산광역시, 해운대구, 좌동, 해운대, 애, 드웰, 오피스텔, 전입, 불가, 실,...","[-0.05811105, -0.043969177, -0.009929976, 0.01..."
78141,34165927,35.167730,129.176674,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,부산광역시 해운대구 좌동,"해운대투모로우오피스텔 3단복층 ♥실사진, 장산역 분♥\r\n\r\n오피스텔 540세...",https://ic.zigbang.com/ic/items/34165927/1.jpg,월세,500,60.0,9.0,부산광역시 해운대구 좌동 해운대투모로우오피스텔 단복층 실사진 장산역 분 오피스텔 세...,"[부산광역시, 해운대구, 좌동, 해운대, 투모로우, 오피스텔, 단복, 층, 실, 사...","[부산광역시, 해운대구, 좌동, 해운대, 투모로우, 오피스텔, 단복, 층, 실, 사...","[-0.022940556, -0.06002051, -0.039477464, 0.02..."
78151,34166032,35.168440,129.176534,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,부산광역시 해운대구 좌동,"해운대 두산위브센티움 복층 ♥실사진, 장산역 3분♥\r\n\r\n두산 위브 센티움 ...",https://ic.zigbang.com/ic/items/34166032/1.jpg,월세,500,38.0,8.0,부산광역시 해운대구 좌동 해운대 두산위브센티움 복층 실사진 장산역 분 두산 위브 센...,"[부산광역시, 해운대구, 좌동, 해운대, 두산, 위브, 센티, 움, 복층, 실, 사...","[부산광역시, 해운대구, 좌동, 해운대, 두산, 위브, 센티, 움, 복층, 실, 사...","[-0.065099575, -0.06351616, 0.0013924167, 0.04..."
80343,34199281,35.384552,129.345293,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,울산시 울주군 서생면,🌹🌹넓고 깔끔합니다🌞🌞 1.5룸정도로 넓고 환한집입니다\r\n해수욕장 도보가능\r\...,https://ic.zigbang.com/ic/items/34199281/1.jpg,월세,300,30.0,5.0,울산시 울주군 서생면 넓고 깔끔합니다 룸정도로 넓고 환한집입니다 해수욕장 도보가능 ...,"[산시, 울주군, 서생면, 룸정도, 집, 해수욕장, 도보, 가능, 원자력, 숙소]","[산시, 울주군, 서생면, 룸정도, 집, 해수욕장, 도보, 가능, 원자력, 숙소]","[-0.06551335, -0.148851, -0.012340089, 0.08909..."
78159,34254089,35.168777,129.176270,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,부산광역시 해운대구 좌동,"해운대 경동윈츠오피스텔 단기입니다. ♥실사진, 장산역 바로앞♥\r\n\r\n경동위...",https://ic.zigbang.com/ic/items/34254089/1.jpg,월세,500,37.0,7.0,부산광역시 해운대구 좌동 해운대 경동윈츠오피스텔 단기입니다 실사진 장산역 바로앞 경...,"[부산광역시, 해운대구, 좌동, 해운대, 윈츠, 오피스텔, 단기, 실, 사진, 장산...","[부산광역시, 해운대구, 좌동, 해운대, 윈츠, 오피스텔, 단기, 실, 사진, 장산...","[-0.042056736, -0.037527688, -0.041252173, 0.0..."
77605,34178692,35.160068,129.151776,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,부산광역시 해운대구 우동,"♥ 풀옵션 단기임대 23,1/7입주 ♥ ★ 해운대 한신 휴플러스 오피스텔 소개 ★...",https://ic.zigbang.com/ic/items/34178692/1.jpg,월세,500,160.0,13.0,부산광역시 해운대구 우동 풀옵션 단기임대 입주 해운대 한신 휴플러스 오피스텔 소개 ...,"[부산광역시, 해운대구, 우동, 풀, 옵션, 단기, 임대, 해운대, 한신, 플러스,...","[부산광역시, 해운대구, 우동, 풀, 옵션, 단기, 임대, 해운대, 한신, 플러스,...","[-0.06063021, -0.032740764, -0.011189294, -0.0..."
78142,34257445,35.168229,129.176042,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,부산광역시 해운대구 좌동,"해운대투모로우 오피스텔 ♥실사진, 장산역 분♥\r\n\r\n오피스텔 570세대 \r...",https://ic.zigbang.com/ic/items/34257445/1.jpg,월세,5000,40.0,9.0,부산광역시 해운대구 좌동 해운대투모로우 오피스텔 실사진 장산역 분 오피스텔 세대 장...,"[부산광역시, 해운대구, 좌동, 해운대, 투모로우, 오피스텔, 실, 사진, 장산역,...","[부산광역시, 해운대구, 좌동, 해운대, 투모로우, 오피스텔, 실, 사진, 장산역,...","[-0.03091731, -0.0478947, -0.04462324, 0.03015..."
98799,34023033,35.149523,129.115460,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items

In [38]:
for i in recommend['doc']:
    print(i)

부산광역시 해운대구 우동 더블역세권 분리형 방 거 바닷가 도보권 최상의입지 해운대역 중동역 더블역세권의 해운대에서도 가장 좋은 입지를 자랑하는 곳입니다 해운대 시장 구청 우체국 해리단길 구남로 해운대 해수욕장등의 다양한 인프라가 있어 생활이 편리하여 인기가 높은 오피스텔입니다 방과 거실이 분리되어있어 활용도가 높고 인이 거주하시기에 딱 좋은 저렴 월세매물 
부산광역시 해운대구 좌동 해운대애드웰오피스텔입니다 전입불가 실사진 장산역 분 오피스텔 세대 장산역 번 출구 분거리 특징 역세권 귀한 신축 전세입니다 보안시절 비디어폰 설치 주변환경 대형마트 대형 병원 재래 시장 롯데시네마 매가박스 백화점 다양한 카페와 식당 다양한 운동 센터 각종학원 해운대 바닷가 분거리 편리하고 조용하여 살기 좋은 동네 입니다 
부산광역시 해운대구 좌동 해운대투모로우오피스텔 단복층 실사진 장산역 분 오피스텔 세대 장산역 번 출구 분거리 평면 타워 주차 넉넉합니다 특징 바다전망 실내 골프장 개장예정 실내 헬스장 이용가능 보안시절 비디어폰 설치 경비원 계심 주변환경 대형마트 대형 서점 대형 병원 재래 시장 롯대시네마 매가박스 백화점 다양한 카페와 식당 다양한 운동 센터 각종학원 해운대 바닷가 분거리 편리하고 조용하여 살기 좋은 동네 입니다 
부산광역시 해운대구 좌동 해운대 두산위브센티움 복층 실사진 장산역 분 두산 위브 센티움 오피스텔 세대 장산역 번 출구 분거리 타워식 주차입니다 특징 방음이 잘됩니다 공간활용이 뛰어납니다 보안시절 비디어폰 설치 경비원 상주 주변환경 대형마트 대형 병원 재래 시장 롯대시네마 매가박스 백화점 다양한 카페와 식당 다양한 운동 센터 해운대 바닷가 분거리 편리하고 살기 좋은 동네 입니다 
울산시 울주군 서생면 넓고 깔끔합니다 룸정도로 넓고 환한집입니다 해수욕장 도보가능 원자력이 가까운숙소입니다 즉시입주가능합니다 
부산광역시 해운대구 좌동 해운대 경동윈츠오피스텔 단기입니다 실사진 장산역 바로앞 경동위츠오피스텔 세대 장산역 번 출구 분거리 타워 주차 넉넉합니다 특징 전용

In [51]:
sim_scores

[(43853, array([0.858411], dtype=float32)),
 (22736, array([0.8513477], dtype=float32)),
 (23602, array([0.8499069], dtype=float32)),
 (52565, array([0.84770536], dtype=float32)),
 (52494, array([0.8473438], dtype=float32)),
 (23605, array([0.84676266], dtype=float32)),
 (53989, array([0.8457395], dtype=float32)),
 (26869, array([0.8442426], dtype=float32)),
 (49973, array([0.84356785], dtype=float32)),
 (52920, array([0.8426478], dtype=float32)),
 (52921, array([0.8418561], dtype=float32)),
 (53937, array([0.8417082], dtype=float32)),
 (57023, array([0.84140974], dtype=float32)),
 (26565, array([0.84055424], dtype=float32)),
 (54110, array([0.84022486], dtype=float32)),
 (53054, array([0.8399974], dtype=float32)),
 (26395, array([0.83963406], dtype=float32)),
 (26063, array([0.838905], dtype=float32)),
 (57325, array([0.8388336], dtype=float32)),
 (55534, array([0.83858275], dtype=float32)),
 (53250, array([0.8377812], dtype=float32)),
 (54088, array([0.83322906], dtype=float32)),
 (5

In [25]:
cosine_similarities = cosine_similarity(sentence_vectors, [input_vector])

In [26]:
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (102286, 1)


In [27]:
cosine_similarities

array([[0.39189595],
       [0.4190197 ],
       [0.46061915],
       ...,
       [0.45075575],
       [0.48938355],
       [0.5698509 ]], dtype=float32)

In [28]:
data.to_pickle('korea_room_list_vectorized.pkl')

In [17]:
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

In [21]:
euclidean_distances(sentence_vectors, [input_vector])

array([[1.3130363],
       [1.2228631],
       [1.229363 ],
       ...,
       [1.351939 ],
       [1.2805932],
       [1.2460063]], dtype=float32)

In [48]:
data

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,token,text_noun
0,34134003,38.077637,128.618335,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,강원도 양양군 양양읍,신축첫입주 전세 월세 신축첫입주입니다\r\n월세는 2층\r\n전세는 3층부터입니다\...,https://ic.zigbang.com/ic/items/34134003/1.jpg,전세,8000,NaN,7.0,강원도 양양군 양양읍 신축첫입주 전세 월세 신축첫입주입니다 월세는 층 전세는 층부터...,"[강원도, 양양군, 양양읍, 신축, 입주, 전세, 월세, 신축, 입, 주입, 다, ...","[강원도, 양양군, 양양읍, 신축, 입주, 전세, 월세, 신축, 입, 주입, 월세,..."
1,34153813,38.077641,128.618986,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/341...,강원도 양양군 양양읍,1.5룸 큰원룸 풀옵션 신축첫입주 신축첫입주입니다\r\n\r\n전세는 3층 부터입니...,https://ic.zigbang.com/ic/items/34153813/1.jpg,전세,10000,NaN,10.0,강원도 양양군 양양읍 룸 큰원룸 풀옵션 신축첫입주 신축첫입주입니다 전세는 층 부터입...,"[강원도, 양양군, 양양읍, 룸, 크다, 원룸, 풀, 옵션, 신축, 입주, 신축, ...","[강원도, 양양군, 양양읍, 룸, 원룸, 풀, 옵션, 신축, 입주, 신축, 입, 주..."
2,34273845,38.184565,128.601096,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,강원도 속초시 조양동,동해바다가 잘보이는 방1 거실1 구조 전입신고 x\r\n동해바다 속초해수욕장 대포항...,https://ic.zigbang.com/ic/items/34273845/1.jpg,월세,500,70.0,15.0,강원도 속초시 조양동 동해바다가 잘보이는 방 거실 구조 전입신고 동해바다 속초해수욕...,"[강원도, 속초시, 조양동, 동해, 바다, 보이다, 방, 거실, 구조, 전입, 신고...","[강원도, 속초시, 조양동, 동해, 바다, 방, 거실, 구조, 전입, 신고, 동해,..."
3,34252466,38.186523,128.598820,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,강원도 속초시 조양동,조양동 분리형원룸 조양동 분리형원룸\r\n교통과 인프라가 잘갖춰져있으며\r\n주택단...,https://ic.zigbang.com/ic/items/34252466/1.jpg,월세,500,40.0,7.0,강원도 속초시 조양동 조양동 분리형원룸 조양동 분리형원룸 교통과 인프라가 잘갖춰져있...,"[강원도, 속초시, 조양동, 조양동, 분리, 원룸, 조양동, 분리, 원룸, 교통, ...","[강원도, 속초시, 조양동, 조양동, 분리, 원룸, 조양동, 분리, 원룸, 교통, ..."
4,34302547,38.187599,128.594645,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,강원도 속초시 조양동,조양동 신축 풀옵션 분리형 원룸 신축건물입니다\r\n반려동물 사육불가능 합니다\r\...,https://ic.zigbang.com/ic/items/34302547/1.jpg,월세,1000,55.0,5.0,강원도 속초시 조양동 조양동 신축 풀옵션 분리형 원룸 신축건물입니다 반려동물 사육불...,"[강원도, 속초시, 조양동, 조양동, 신축, 풀, 옵션, 분리, 원룸, 신축, 건물...","[강원도, 속초시, 조양동, 조양동, 신축, 풀, 옵션, 분리, 원룸, 신축, 건물..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102281,34203993,33.538358,126.664236,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,제주도 제주시 조천읍,⭐조천읍 함덕리 반려묘 가능한 분리형 원룸 전세⭐ ❤제주 핫한 BEST방❤\r\n\...,https://ic.zigbang.com/ic/items/34203993/1.jpg,전세,10000,NaN,5.0,제주도 제주시 조천읍 조천읍 함덕리 반려묘 가능한 분리형 원룸 전세 제주 핫한 방 ...,"[제주도, 제주시, 조천읍, 조천읍, 함덕, 리, 반려, 묘, 분리, 원룸, 전세,...","[제주도, 제주시, 조천읍, 조천읍, 함덕, 리, 반려, 묘, 분리, 원룸, 전세,..."
102282,34309621,33.537858,126.663626,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/343...,제주도 제주시 조천읍,🍒🍒테라스있는오픈형원룸&풀옵션 🍒🍒테라스있는오픈형원룸&풀옵션\r\n\r\n전세대출가...,https://ic.zigbang.com/ic/items/34309621/1.jpg,전세,9000,NaN,3.0,제주도 제주시 조천읍 테라스있는오픈형원룸 풀옵션 테라스있는오픈형원룸 풀옵션 전세대출...,"[제주도, 제주시, 조천읍, 테라스, 오픈, 원룸, 풀, 옵션, 테라스, 오픈, 원...","[제주도, 제주시, 조천읍, 테라스, 오픈, 원룸, 풀, 옵션, 테라스, 오픈, 원..."
102283,34269795,33.540079,126.676017,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,제주도 제주시 조천읍,🍒🍒함덕해수욕장인근&풀옵션원룸&반려동물X 🍒🍒함덕해수욕장인근&풀옵션원룸&반려동물X\...,https://ic.zigbang.com/ic/items/34269795/1.jpg,월세,100,54.0,1.0,제주도 제주시 조천읍 함덕해수욕장인근 풀옵션원룸 반려동물 함덕해수욕장인근 풀옵션원룸...,"[제주도, 제주시, 조천읍, 함덕, 해수욕장, 인근, 풀, 옵션, 원룸, 반려동물,...","[제주도, 제주시, 조천읍, 함덕, 해수욕장, 인근, 풀, 옵션, 원룸, 반려동물,..."
102284,34296056,33.541841,126.662953,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/342...,제주도 제주시 조천읍,++제주시 함덕 해수욕장 부근 깔끔한 원룸 매물++ ✔상세설명✔\r\n\r\n \...,https://ic.zigbang.com/ic/items/34296056/1.jpg,월세,100,60.0,5.0,제주도 제주시 조천읍 제주시 함덕 해수욕장 부근 깔끔한 원룸 매물 상세설명 넓고 아...,"[제주도, 제주시, 조천읍, 제주시, 함덕, 해수욕장, 부근, 원룸, 매물, 상세,...","[제주도, 제주시, 조천읍, 제주시, 함덕, 해수욕장, 부근, 원룸, 매물, 상세,..."
